In [1]:
# to use virtual env
# python3 -m venv venv
# source venv/bin/activate

!pip3 install --upgrade pip
!pip3 install ultralytics opencv-python
!pip3 install pandas
!pip3 install albumentations
!pip3 install pytesseract

  Using cached pip-25.2-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
  Using cached ultralytics-8.3.174-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.3.2-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached matplotlib-3.10.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached pillow-11.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.16.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached torch-2.7.1-cp312-none-macosx_11_0_arm64.whl.metadata (29 kB)
  Using cached torchvision-0.22.1-cp312-cp312-ma

In [ ]:
# download the dataset: https://www.kaggle.com/datasets/andrewmvd/car-plate-detection?resource=download
# rename the folder: "dataset"

In [1]:
import os
import shutil
import random
import time
import pandas as pd
from ultralytics import YOLO
import xml.etree.ElementTree as ET
import torch
import cv2
import pytesseract
import json
from collections import defaultdict



In [2]:
# process the annotations in YOLO text format
# split the data into train 70%, val 20% and test 10%

random.seed(42)

image_dir = 'dataset/images'
label_dir = 'dataset/annotations'

out_base = 'dataset_split'

def convert_xml_to_yolo(xml_path, txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find('size')
    width = float(size.find('width').text)
    height = float(size.find('height').text)

    lines = []
    for obj in root.findall('object'):
        class_id = 0 

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        w = (xmax - xmin) / width
        h = (ymax - ymin) / height

        lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(txt_path, 'w') as f:
        f.write("\n".join(lines))

image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
base_names = [os.path.splitext(f)[0] for f in image_files]

random.shuffle(base_names)
total = len(base_names)
train_end = int(0.7 * total)
val_end = int(0.9 * total)

splits = {
    'train': base_names[:train_end],
    'val': base_names[train_end:val_end],
    'test': base_names[val_end:]
}

for split in ['train', 'val', 'test']:
    os.makedirs(f'{out_base}/images/{split}', exist_ok=True)
    os.makedirs(f'{out_base}/labels/{split}', exist_ok=True)

for split, names in splits.items():
    for name in names:
        img_src = os.path.join(image_dir, f'{name}.png')
        lbl_src = os.path.join(label_dir, f'{name}.xml')

        img_dst = os.path.join(out_base, 'images', split, f'{name}.png')
        txt_dst = os.path.join(out_base, 'labels', split, f'{name}.txt')

        if os.path.exists(img_src):
            shutil.copy(img_src, img_dst)

        if os.path.exists(lbl_src):
            convert_xml_to_yolo(lbl_src, txt_dst)

print("Files split, grouped, and labels converted to YOLO TXT format")


Files split, grouped, and labels converted to YOLO TXT format


In [3]:
if torch.backends.mps.is_available():
    device = "mps"

elif torch.cuda.is_available():        
    device = "cuda"

else:
    device = "cpu"

print(f"Using device: {device}")


Using device: mps


In [4]:
# models = ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt"]
# data_yaml = "dataset_split/data.yaml"
# epochs = 5

# results = []

# for model_name in models:
#     print(f"\nTraining {model_name}...")
#     model = YOLO(model_name)
    
#     start_time = time.time()
#     metrics = model.train(data=data_yaml, epochs=epochs)
#     end_time = time.time()
#     train_time = round(end_time - start_time, 2)
    
#     map50 = metrics.results_dict.get("metrics/mAP50(B)", None)
#     recall = metrics.results_dict.get("metrics/recall(B)", None)
#     precision = metrics.results_dict.get("metrics/precision(B)", None)
#     f1 = metrics.results_dict.get("metrics/f1(B)", None)
    
#     results.append({
#         "Model": model_name,
#         "Precision": round(precision, 4) if precision else None,
#         "Recall": round(recall, 4) if recall else None,
#         "F1 Score": round(f1, 4) if f1 else None,
#         "mAP50": round(map50, 4) if map50 else None,
#         "Training Time (s)": train_time
#     })

# df = pd.DataFrame(results)
# df.to_csv("yolo_model_comparison.csv", index=False)
# display(df)


In [6]:
# train and validate model
# from the table above I could see yolo v8 small performs better

model = YOLO("yolov8m.pt")
model.train(data="dataset_split/data.yaml", name="model_blur",
        epochs=10,
        batch=8,
        device = "cpu"
        )
metrics = model.val(data="dataset_split/data.yaml")
print(metrics)


Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_split/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=model_blur, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, 

train: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/train.cache... 303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 303/303 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 826.9±98.9 MB/s, size: 559.0 KB)



val: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/val.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]

Plotting labels to /Users/neslyn/Desktop/dl_project/runs/detect/model_blur/labels.jpg... 


...


10 epochs completed in 1.242 hours.
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/model_blur/weights/last.pt, 52.0MB
Optimizer stripped from /Users/neslyn/Desktop/dl_project/runs/detect/model_blur/weights/best.pt, 52.0MB

Validating /Users/neslyn/Desktop/dl_project/runs/detect/model_blur/weights/best.pt...
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:51<00:00,  8.61s/it]


                   all         86         92      0.929       0.75      0.893      0.503
Speed: 0.9ms preprocess, 587.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to /Users/neslyn/Desktop/dl_project/runs/detect/model_blur
Ultralytics 8.3.174 🚀 Python-3.12.8 torch-2.7.1 CPU (Apple M1)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 589.5±225.4 MB/s, size: 409.8 KB)


val: Scanning /Users/neslyn/Desktop/dl_project/dataset_split/labels/val.cache... 86 images, 0 backgrounds, 0 corrupt: 100%|██████████| 86/86 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:20<00:00,  1.89s/it]


                   all         86         92      0.929       0.75      0.891      0.504
Speed: 0.4ms preprocess, 231.0ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /Users/neslyn/Desktop/dl_project/runs/detect/model_blur2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x127a96060>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.0300

In [11]:
model = YOLO("runs/detect/model_blur/weights/best.pt")
results = model.predict(source="test.mp4", conf=0.4, save=True)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1588) /Users/neslyn/Desktop/dl_project/test.mp4: 640x384 1 licence, 185.4ms
video 1/1 (frame 2/1588) /Users/neslyn/Desktop/dl_project/test.mp4: 640x384 1 licence, 154.7ms
video 1/1 (frame 3/1588) /Users/neslyn/Desktop/dl_project/test.mp4: 640x384 1 licence, 151.5ms
video 1/1 (frame 4/1588) /Users/neslyn/Desktop/dl_project/test.mp4: 640x384 1 licence, 191.8ms
video 1/1 (frame 5/1588) /Users/neslyn/Desktop/dl_project/test.mp4: 640x384 

In [13]:
vehicle_model = YOLO("yolov8s.pt")  # Detects vehicles
plate_model = YOLO("runs/detect/model_blur/weights/best.pt")  # Detects license plates

# Video input/output setup
input_path = "test1.mp4"
cap = cv2.VideoCapture(input_path)
assert cap.isOpened(), f"Failed to open video file: {input_path}"

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))

# Folder to store detected plate crops
output_folder = "license_plates"
os.makedirs(output_folder, exist_ok=True)

plate_counter = defaultdict(int)  # ✨ For image file versioning per timestamp

frame_index = 0
vehicle_class_ids = [2, 3, 5, 7]  # car, motorcycle, bus, truck

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Detect vehicles
    vehicle_results = vehicle_model.predict(source=frame, conf=0.4, verbose=False)
    vehicle_boxes = []

    for result in vehicle_results:
        for box, cls in zip(result.boxes.xyxy.cpu().numpy(), result.boxes.cls.cpu().numpy()):
            if int(cls) in vehicle_class_ids:
                x1, y1, x2, y2 = map(int, box)
                x1, y1 = max(0, x1), max(0, y1)
                x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
                vehicle_boxes.append((x1, y1, x2, y2))

    # Step 2: For each vehicle box, detect license plates
    for x1, y1, x2, y2 in vehicle_boxes:
        vehicle_roi = frame[y1:y2, x1:x2]
        if vehicle_roi.size == 0:
            continue

        plate_results = plate_model.predict(source=vehicle_roi, conf=0.4, verbose=False)

        for result in plate_results:
            if result.boxes is None or len(result.boxes) == 0:
                continue

            for box in result.boxes.xyxy.cpu().numpy():
                px1, py1, px2, py2 = map(int, box)
                abs_x1 = x1 + px1
                abs_y1 = y1 + py1
                abs_x2 = x1 + px2
                abs_y2 = y1 + py2

                abs_x1, abs_y1 = max(0, abs_x1), max(0, abs_y1)
                abs_x2, abs_y2 = min(frame.shape[1], abs_x2), min(frame.shape[0], abs_y2)

                plate_roi = frame[abs_y1:abs_y2, abs_x1:abs_x2]
                if plate_roi.size == 0:
                    continue

                # Compute timestamp
                timestamp_seconds = frame_index / fps
                hours = int(timestamp_seconds // 3600)
                minutes = int((timestamp_seconds % 3600) // 60)
                seconds = int(timestamp_seconds % 60)
                timestamp_label = f"{hours:02}:{minutes:02}:{seconds:02}"
                timestamp_str = f"{hours:02}_{minutes:02}_{seconds:02}"
                print(f"processing time stamp:{timestamp_label}")
   
                # Increment counter for this timestamp
                plate_counter[timestamp_str] += 1
                suffix = plate_counter[timestamp_str]

                # Save cropped image with suffix
                filename = os.path.join(output_folder, f"{timestamp_str}_{suffix}.jpg")
                cv2.imwrite(filename, plate_roi)

                # Blur plate in original frame
                frame[abs_y1:abs_y2, abs_x1:abs_x2] = cv2.GaussianBlur(plate_roi, (35, 35), 0)

                # Draw red bounding box
                cv2.rectangle(frame, (abs_x1, abs_y1), (abs_x2, abs_y2), (0, 0, 255), 3)

                # Put filename above box
                label = f"{timestamp_str}_{suffix}.jpg"
                text_position = (abs_x1, max(0, abs_y1 - 10))
                cv2.putText(frame, label, text_position,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2, cv2.LINE_AA)


    out.write(frame)
    frame_index += 1

cap.release()
out.release()


print("Blurred video saved as output.mp4")
print(f"Plate crops saved in folder: {output_folder}")


processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processing time stamp:00:00:00
processi

KeyboardInterrupt: 